In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets peft accelerate bitsandbytes trl safetensors torch --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 326.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 316.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 267.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 281.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 302.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 201.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 107.1 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
from functools import partial
from datasets import Dataset
from transformers import TrainingArguments
import time
import transformers
from sklearn.model_selection import train_test_split
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

column_names = ['K', 'RK', 'E']
# base_path = '/content/drive/My Drive/Colab Notebooks/CSCI 564 NLP'
# df = pd.read_csv(f'{base_path}/hindi_data/romanized_hindi_english_paper.csv', names=column_names,nrows=8000)

path = '/content/drive/MyDrive/Project_544/data/korean_data/korean_romanized_english.csv'

df = pd.read_csv(path, names=column_names, nrows=5000)

In [4]:
# Hugging Face model name
#model_name = "microsoft/phi-2" # not the larger version - need to look into this
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
use_flash_attention = False


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16
)

model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [6]:
# for RH to E
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

In [7]:
# for H to E
train_df2, val_df2 = train_df.copy(), val_df.copy()
val_df2, test_df2 = val_df.copy(), test_df.copy()

In [8]:
# for RH+H to E
train_df3, val_df3 = train_df.copy(), val_df.copy()
val_df3, test_df3 = val_df.copy(), test_df.copy()

In [9]:
results = {'Languages': [],
           'Avg Meteor' : []}
           #'Corpus BLEU' : []}

# **Korean to English**

In [10]:
train_df, val_df = train_df, val_df
val_df, test_df = val_df, test_df

In [11]:

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    print(batch)
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


_preprocessing_function = partial(preprocess_batch, max_length=1024, tokenizer=tokenizer)
# Convert train_df and val_df into JSON format
# train_dataset = [create_json_record(row, "train") for _, row in train_df.iterrows()]
# val_dataset = [create_json_record(row, "val") for _, row in val_df.iterrows()]


def create_prompt(row):
    return f"Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: {row['K']} \nEnglish: {row['E']}"

# Apply the function to each row and create a new 'text' column
train_df['text'] = train_df.apply(create_prompt, axis=1)
val_df['text'] = val_df.apply(create_prompt, axis=1)

columns_to_drop = ['K', 'RK','E']
train_df.drop(columns=columns_to_drop, inplace=True)
val_df.drop(columns=columns_to_drop, inplace=True)

train_dataset = Dataset.from_pandas(train_df)
val_dataset= Dataset.from_pandas(val_df)


In [12]:
train_dataset = train_dataset.map(
        _preprocessing_function,
        batched=True,
    )

Map:   0%|          | 0/3651 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 먹어 보니까 볶음밥이 훨씬 제 입에 맞는 것 같아요. \nEnglish: Now that I've tried it, I think fried rice is better for my taste.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 스티브 씨도 오늘 동호회 모임에 갈 거지요? \nEnglish: Mr. Steve, are you going to the club meeting today?', "Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 몰랐어요. 그럼 다음에 만납시다. \nEnglish: I didn't know. Then, let's meet next time.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 저는 고기를 좋아합니다. 그래서 자주 먹습니다. \nEnglish: I like meat. So I eat meat often.', "Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 한번 생각해 볼게. \nEnglish: I'll

In [13]:
val_dataset = val_dataset.map(
        _preprocessing_function,
    batched=True,
    )

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 혹시 남은 음식 포장이 되나요? 배가 너무 불러서 다 못 먹을 것 같아요. \nEnglish: Is it possible to take our leftovers as a to-go? My stomach is so full that I won't be able to eat the rest.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 어떤 점이 힘들었습니까? \nEnglish: What was the difficult part?', 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 심리학 수업 발표 때문에 조모임을 해야 하는데 내일 시간 있어? \nEnglish: Since we have a group presentation for psychology, we have to meet together, do you have time tomorrow?', 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: 그래요? 저는 이렇게 짧은 치마는 처음 입어 봐요. \nEnglish: Really? This is my first time to try this short skirt on.', 'Translate the following 

In [14]:
base_path = "/content/drive/My Drive/CSCI544ProjOutput/"
output_dir = f'{base_path}/peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=50,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=False,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
print(type(train_dataset['text'][0]))

<class 'str'>


In [16]:
peft_trainer.train()

Step,Training Loss,Validation Loss
25,1.537100,1.195430
50,0.955800,1.100056


TrainOutput(global_step=50, training_loss=1.246475715637207, metrics={'train_runtime': 103.0775, 'train_samples_per_second': 1.94, 'train_steps_per_second': 0.485, 'total_flos': 112250566471680.0, 'train_loss': 1.246475715637207, 'epoch': 0.05})

In [17]:
# from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Assume test_data is a list of tuples: (source_sentence, reference_translation)

sources = test_df['K'].to_list()
references = test_df['E'].to_list()

#sources = [sources[0]]
#references = [references[0]]

s = []
r = []

for i in range(len(sources)):

    inputs = tokenizer(f'''Translate the following sentences from Korean to English. The output should be in English and no other language. \nKorean: {sources[i]} \nEnglish:''', return_tensors="pt",)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=20)
        text = tokenizer.batch_decode(outputs)[0]
        #print(text)

    #print(text)
    t = text.split("\nEnglish: ")
    t = t[1]. split("\n")[0]

    """
    print("t")
    print(t)
    print(t.split())
    print(references[i].split())
    print("---")
    """

    s.append(t.split())
    # translations.append(reference.split())
    r.append(references[i].split())  # Note: reference must be a list of lists for corpus_bleu

In [18]:
print(s)
print(r)

[['I', 'have', 'been', 'eating', 'like', 'this', 'for', 'the', 'past', 'two', 'weeks.'], ['You', 'look', 'handsome.'], ['I', 'feel', 'stressed', 'a', 'lot,', 'but', 'my', 'headache', 'is', 'getting', 'worse.', 'Do', 'you', 'have', 'any', 'idea'], ['Yes,', 'I', 'know', 'all', 'the', 'songs', 'of', 'the', 'singer.'], ['Yes,', "that's", 'right.', 'A', 'small', 'tree', 'is', 'not', 'safe', 'in', 'the', 'wind.', 'But', 'a', 'big'], ['Yes,', "that's", 'right.', 'The', 'insurance', 'company', 'requires', 'the', 'documents', 'before', 'submitting', 'them.', 'I'], ['Thank', 'you.', 'Then,', 'I', 'will', 'put', 'the', 'left', 'foot', 'on', 'the', 'ground', 'and', 'step', 'on', 'the', 'right', 'foot'], ['Yesterday', 'I', 'saw', 'the', 'news.', 'Now', 'we', 'can', 'also', 'go', 'on', 'a', 'trip', 'to', 'the', 'universe.'], ['Please', 'put', 'it', 'here.', 'This', 'is', 'our', 'house.'], ['The', 'speed', 'of', 'light', 'is', 'much', 'faster', 'than', 'the', 'speed', 'of', 'sound,', 'so', 'the', 'so

In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [20]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [21]:
from evaluate import load
meteor_metric = load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [22]:
hypothesis = [' '.join(i) for i in s]
reference = [' '.join(i) for i in r]
print(len(hypothesis))
print(len(reference))
filtered_data = [(h, r) for h, r in zip(hypothesis, reference) if h and r]

# If you want to separate the filtered data into hypothesis and reference lists again
filtered_hypothesis = [pair[0] for pair in filtered_data]
filtered_reference = [pair[1] for pair in filtered_data]
print(len(filtered_hypothesis))
print(len(filtered_reference))
# bleurt_results = bleurt_metric.compute(predictions=hypothesis, references=reference)
meteor_scores = meteor_metric.compute(predictions=filtered_hypothesis, references=filtered_reference)
# meteor_results = [round(v, 4) for v in meteor_scores["meteor"]]
# avg = sum(meteor_results)/len(meteor_results)
avg= meteor_scores["meteor"]

457
457
457
457


In [23]:
avg

0.36388504102856967

In [24]:
results['Languages'].append('Korean -> English')
results['Avg Meteor'].append(avg)
print(avg)

0.36388504102856967


#**Romanized Korean to English**

In [25]:
train_df, val_df = train_df2, val_df2
val_df, test_df = val_df2, test_df2

In [26]:

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    print(batch)
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


_preprocessing_function = partial(preprocess_batch, max_length=1024, tokenizer=tokenizer)
# Convert train_df and val_df into JSON format
# train_dataset = [create_json_record(row, "train") for _, row in train_df.iterrows()]
# val_dataset = [create_json_record(row, "val") for _, row in val_df.iterrows()]


def create_prompt(row):
    return f"Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: {row['RK']} \nEnglish: {row['E']}"

# Apply the function to each row and create a new 'text' column
train_df['text'] = train_df.apply(create_prompt, axis=1)
val_df['text'] = val_df.apply(create_prompt, axis=1)

columns_to_drop = ['K', 'RK','E']
train_df.drop(columns=columns_to_drop, inplace=True)
val_df.drop(columns=columns_to_drop, inplace=True)

train_dataset = Dataset.from_pandas(train_df)
val_dataset= Dataset.from_pandas(val_df)


In [27]:
train_dataset = train_dataset.map(
        _preprocessing_function,
        batched=True,
    )

Map:   0%|          | 0/3651 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: meogeo bonikka bokkeumbabi hwolssin je ibe matneun geot gatayo. \nEnglish: Now that I've tried it, I think fried rice is better for my taste.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: seutibeu ssido oneul donghohoe moime gal geojiyo? \nEnglish: Mr. Steve, are you going to the club meeting today?', "Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: molrasseoyo. geureom daeume mannapsida. \nEnglish: I didn't know. Then, let's meet next time.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: jeoneun gogireul joahapnida. geuraeseo jaju meokseupnida. \nEnglish: I like meat. So I eat meat often.', "Translate the following sentences fro

In [28]:
val_dataset = val_dataset.map(
        _preprocessing_function,
    batched=True,
    )

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: hoksi nameun eumsik pojangi doenayo? baega neomu bulreoseo da mot meogeul geot gatayo. \nEnglish: Is it possible to take our leftovers as a to-go? My stomach is so full that I won't be able to eat the rest.", 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: eotteon jeomi himdeureotseupnikka? \nEnglish: What was the difficult part?', 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: simrihak sueop balpyo ttaemune jomoimeul haeya haneunde naeil sigan isseo? \nEnglish: Since we have a group presentation for psychology, we have to meet together, do you have time tomorrow?', 'Translate the following sentences from Korean to English. The output should be in English and no other language.\nKorean: geuraeyo? jeoneun ir

In [29]:
base_path = "/content/drive/My Drive/CSCI544ProjOutput/"
output_dir = f'{base_path}/peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=50,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=False,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
print(type(train_dataset['text'][0]))

<class 'str'>


In [31]:
peft_trainer.train()

Step,Training Loss,Validation Loss
25,2.787200,2.432630
50,1.865900,2.412821


TrainOutput(global_step=50, training_loss=2.326535491943359, metrics={'train_runtime': 99.6393, 'train_samples_per_second': 2.007, 'train_steps_per_second': 0.502, 'total_flos': 92313571983360.0, 'train_loss': 2.326535491943359, 'epoch': 0.05})

In [32]:
# model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/peft-dialogue-summary-training-1712294967/checkpoint-375')
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/CSCI544ProjOutput/peft-dialogue-summary-training-1712294967/checkpoint-375',trust_remote_code=True)

In [33]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Assume test_data is a list of tuples: (source_sentence, reference_translation)

sources = test_df['RK'].to_list()
references = test_df['E'].to_list()

#sources = [sources[0]]
#references = [references[0]]

s = []
r = []

for i in range(len(sources)):

    inputs = tokenizer(f'''Translate the following sentences from Korean to English. The output should be in English and no other language. \nKorean: {sources[i]} \nEnglish:''', return_tensors="pt",)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=20)
        text = tokenizer.batch_decode(outputs)[0]
        #print(text)

    #print(text)
    t = text.split("\nEnglish: ")
    t = t[1]. split("\n")[0]

    """
    print("t")
    print(t)
    print(t.split())
    print(references[i].split())
    print("---")
    """

    s.append(t.split())
    # translations.append(reference.split())
    r.append(references[i].split())  # Note: reference must be a list of lists for corpus_bleu

In [34]:
print(s)
print(r)

[["I'm", 'going', 'to', 'the', 'park', 'today.'], ['Yes,', "I'm", 'fine.'], ["I'm", 'going', 'to', 'the', 'park', 'today.', "I'm", 'going', 'to', 'go', 'to', 'the', 'park.'], ['Can', 'you', 'tell', 'me', 'how', 'to', 'make', 'a', 'salad?'], ['Yes,', 'I', 'understand.', "I'll", 'go', 'to', 'the', 'store', 'and', 'buy', 'some', 'snacks.'], ['Yes,', "I'll", 'go', 'to', 'the', 'park.', "I'll", 'take', 'my', 'dog', 'there.'], ["I'm", 'sorry.', "I'll", 'make', 'it', 'right.'], ['Can', 'you', 'tell', 'me', 'how', 'to', 'make', 'the', 'dish?', 'I', 'want', 'to', 'try', 'it.'], ["I'm", 'going', 'to', 'the', 'park.', "I'll", 'go', 'to', 'the', 'park.'], ['I', 'have', 'a', 'lot', 'of', 'things', 'to', 'do.', 'I', 'have', 'to', 'go', 'to', 'the', 'store', 'and', 'buy', 'some', 'things'], ["I'm", 'sorry,', 'but', 'I', "don't", 'understand.', "I'm", 'not', 'sure', 'what', 'you', 'mean'], ["I'm", 'going', 'to', 'the', 'market.'], ["I'm", 'going', 'to', 'the', 'park.'], ["I'm", 'going', 'to', 'the', '

In [35]:
hypothesis = [' '.join(i) for i in s]
reference = [' '.join(i) for i in r]
print(len(hypothesis))
print(len(reference))
filtered_data = [(h, r) for h, r in zip(hypothesis, reference) if h and r]

# If you want to separate the filtered data into hypothesis and reference lists again
filtered_hypothesis = [pair[0] for pair in filtered_data]
filtered_reference = [pair[1] for pair in filtered_data]
print(len(filtered_hypothesis))
print(len(filtered_reference))
# bleurt_results = bleurt_metric.compute(predictions=hypothesis, references=reference)
meteor_scores = meteor_metric.compute(predictions=filtered_hypothesis, references=filtered_reference)
# meteor_results = [round(v, 4) for v in meteor_scores["meteor"]]
# avg = sum(meteor_results)/len(meteor_results)
avg= meteor_scores["meteor"]

457
457
457
457


In [36]:
results['Languages'].append('Romanized Korean -> English')
results['Avg Meteor'].append(avg)
print(avg)

0.1479645151059336


# **RK + K to English**

In [37]:
train_df, val_df = train_df3, val_df3
val_df, test_df = val_df3, test_df3

In [38]:

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    print(batch)
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


_preprocessing_function = partial(preprocess_batch, max_length=1024, tokenizer=tokenizer)
# Convert train_df and val_df into JSON format
# train_dataset = [create_json_record(row, "train") for _, row in train_df.iterrows()]
# val_dataset = [create_json_record(row, "val") for _, row in val_df.iterrows()]


def create_prompt(row):
    return f"Translate the following sentences from Korean to English. The output should be in English and no other language. \Korean: {row['RK']} \Korean: {row['K']} \nEnglish: {row['E']}"

# Apply the function to each row and create a new 'text' column
train_df['text'] = train_df.apply(create_prompt, axis=1)
val_df['text'] = val_df.apply(create_prompt, axis=1)

columns_to_drop = ['K', 'RK','E']
train_df.drop(columns=columns_to_drop, inplace=True)
val_df.drop(columns=columns_to_drop, inplace=True)

train_dataset = Dataset.from_pandas(train_df)
val_dataset= Dataset.from_pandas(val_df)


In [39]:
train_dataset = train_dataset.map(
        _preprocessing_function,
        batched=True,
    )

Map:   0%|          | 0/3651 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: meogeo bonikka bokkeumbabi hwolssin je ibe matneun geot gatayo. \\Korean: 먹어 보니까 볶음밥이 훨씬 제 입에 맞는 것 같아요. \nEnglish: Now that I've tried it, I think fried rice is better for my taste.", 'Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: seutibeu ssido oneul donghohoe moime gal geojiyo? \\Korean: 스티브 씨도 오늘 동호회 모임에 갈 거지요? \nEnglish: Mr. Steve, are you going to the club meeting today?', "Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: molrasseoyo. geureom daeume mannapsida. \\Korean: 몰랐어요. 그럼 다음에 만납시다. \nEnglish: I didn't know. Then, let's meet next time.", 'Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: jeoneun gogireul joahapnida. geurae

In [40]:
val_dataset = val_dataset.map(
        _preprocessing_function,
    batched=True,
    )

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

{'text': ["Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: hoksi nameun eumsik pojangi doenayo? baega neomu bulreoseo da mot meogeul geot gatayo. \\Korean: 혹시 남은 음식 포장이 되나요? 배가 너무 불러서 다 못 먹을 것 같아요. \nEnglish: Is it possible to take our leftovers as a to-go? My stomach is so full that I won't be able to eat the rest.", 'Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: eotteon jeomi himdeureotseupnikka? \\Korean: 어떤 점이 힘들었습니까? \nEnglish: What was the difficult part?', 'Translate the following sentences from Korean to English. The output should be in English and no other language. \\Korean: simrihak sueop balpyo ttaemune jomoimeul haeya haneunde naeil sigan isseo? \\Korean: 심리학 수업 발표 때문에 조모임을 해야 하는데 내일 시간 있어? \nEnglish: Since we have a group presentation for psychology, we have to meet together, do you have time tomorrow?', 'Translate the fol

In [41]:
base_path = "/content/drive/My Drive/CSCI544ProjOutput/"
output_dir = f'{base_path}/peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=50,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=False,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
print(type(train_dataset['text'][0]))

<class 'str'>


In [43]:
peft_trainer.train()

Step,Training Loss,Validation Loss
25,1.915300,1.731445
50,1.365100,1.722954


TrainOutput(global_step=50, training_loss=1.6401774597167968, metrics={'train_runtime': 152.8099, 'train_samples_per_second': 1.309, 'train_steps_per_second': 0.327, 'total_flos': 151564702248960.0, 'train_loss': 1.6401774597167968, 'epoch': 0.05})

In [44]:
# model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/peft-dialogue-summary-training-1712294967/checkpoint-375')
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/CSCI544ProjOutput/peft-dialogue-summary-training-1712294967/checkpoint-375',trust_remote_code=True)

In [45]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# Assume test_data is a list of tuples: (source_sentence, reference_translation)

rk = test_df['RK'].to_list()
k = test_df['K'].to_list()
references = test_df['E'].to_list()

#sources = [sources[0]]
#references = [references[0]]

s = []
r = []

for i in range(len(rk)):

    inputs = tokenizer(f'''Translate the following sentences from Hindi to English. The output should be in English and no other language. \nKorean: {rk[i]} \Korean: {k[i]} \nEnglish:''', return_tensors="pt",)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=20)
        text = tokenizer.batch_decode(outputs)[0]
        #print(text)

    #print(text)
    t = text.split("\nEnglish: ")
    t = t[1]. split("\n")[0]

    """
    print("t")
    print(t)
    print(t.split())
    print(references[i].split())
    print("---")
    """

    s.append(t.split())
    # translations.append(reference.split())
    r.append(references[i].split())  # Note: reference must be a list of lists for corpus_bleu

In [46]:
hypothesis = [' '.join(i) for i in s]
reference = [' '.join(i) for i in r]
print(len(hypothesis))
print(len(reference))
filtered_data = [(h, r) for h, r in zip(hypothesis, reference) if h and r]

# If you want to separate the filtered data into hypothesis and reference lists again
filtered_hypothesis = [pair[0] for pair in filtered_data]
filtered_reference = [pair[1] for pair in filtered_data]
print(len(filtered_hypothesis))
print(len(filtered_reference))
# bleurt_results = bleurt_metric.compute(predictions=hypothesis, references=reference)
meteor_scores = meteor_metric.compute(predictions=filtered_hypothesis, references=filtered_reference)
# meteor_results = [round(v, 4) for v in meteor_scores["meteor"]]
# avg = sum(meteor_results)/len(meteor_results)
avg= meteor_scores["meteor"]

457
457
457
457


In [47]:
results['Languages'].append('Korean + Romanized Korean -> English')
results['Avg Meteor'].append(avg)
print(avg)

0.3418867010892172


# **Results**

In [48]:
results_df = pd.DataFrame(results)
results_df


,Languages,Avg Meteor
0,Korean -> English,0.363885
1,Romanized Korean -> English,0.147965
2,Korean + Romanized Korean -> English,0.341887
